<a href="https://colab.research.google.com/github/Vickymuthu373/LawBot_Fine_Tuned_LLM_and_RAG_Model/blob/main/LLM_LLT_RA2412044015051.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For dataset prep, LLM fine-tuning, and RAG
!pip install pandas scikit-learn transformers peft unsloth sentence-transformers faiss-cpu langchain llama-index streamlit gradio


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# Adjust file paths as needed if using Google Drive
qa_files = ['/content/constitution_qa.json', '/content/crpc_qa.json', '/content/ipc_qa.json']
dataset = []
for file in qa_files:
    with open(file, 'r') as f:
        dataset += json.load(f)

cleaned = []
for record in dataset:
    q, a, src = record['question'], record['answer'], record.get('source', 'Unknown')
    cleaned.append({'instruction': q, 'output': a, 'source': src})

df = pd.DataFrame(cleaned).drop_duplicates()
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_json('/content/lawbot_train.jsonl', orient='records', lines=True)
val_df.to_json('/content/lawbot_val.jsonl', orient='records', lines=True)

print(f"Training size: {len(train_df)}, Validation size: {len(val_df)}")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

lora_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, lora_config)

# Minimalistic dataset loader for demonstration
import datasets
train_data = datasets.load_dataset('json', data_files='/content/lawbot_train.jsonl', split='train')
eval_data = datasets.load_dataset('json', data_files='/content/lawbot_val.jsonl', split='train')

def tokenize_function(example):
    return tokenizer(example["instruction"], truncation=True)
tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_eval = eval_data.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=10,
    output_dir="/content/output",
    logging_dir="/content/logs",
    report_to='none'
)
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    args=training_args
)
# Uncomment below after checking memory limits!
# trainer.train()


In [ ]:
# Sample Indian Legal Text Chunks for LawBot RAG
law_chunks = [
    # IPC Sections
    "Section 81 of IPC: Act likely to cause harm, but done without criminal intent, and to prevent other harm. Nothing is an offence by reason of any harm which it may cause to a person who has consented to suffer that harm. Section 81 defines act, omission or thing which is likely to cause harm, but done without criminal intent and to prevent other harm.",

    "Section 82 of IPC: Nothing is an offence which is done by a person who, at the time of doing it, by reason of immaturity of understanding, is incapable of knowing either the nature of the act, or that he is doing what is wrong or contrary to law.",

    "Section 83 of IPC: Nothing is an offence by reason of any harm which it may cause to a person above eighteen years of age who has consented to suffer that harm, or by reason of any harm which it may cause to a person who is incapable of knowing the nature of the act or that such person is doing what is wrong.",

    "Section 84 of IPC: Nothing is an offence which is done by a person who, at the time of doing it, by reason of unsoundness of mind, is incapable of knowing the nature of the act, or that he is doing what is wrong or contrary to law.",

    "Section 52 of IPC: 'Wrongful gain' is every person's wrongful conveyance or loss of property, and 'wrongful loss' is the opposites. Whoever commits theft in respect of property shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine.",

    "Section 378 of IPC (Theft): Whoever, intending to take dishonestly any movable property out of the possession of any person without that person's consent, moves that property in order by that movement to effect his purpose, is said to commit theft.",

    "Section 420 of IPC (Cheating): Whoever cheats and thereby dishonestly induces the person deceived to deliver any property to any person, or to make, alter or destroy the whole or any part of a valuable security, or anything which is signed or sealed, and which is capable of being converted into a valuable security, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine.",

    "Section 498A of IPC (Cruelty by Husband or In-laws): Whoever, being the husband or the relative of the husband of a woman, subjects such woman to cruelty shall be punished with imprisonment for a term which may extend to three years and shall also be liable to fine.",

    # CrPC Sections
    "Section 41 of CrPC (When Police may effect Arrest without Warrant): An officer in charge of a police station shall, when investigating a case, have the authority to arrest any person against whom a reasonable complaint has been made, or credible information has been received, or reasonable suspicion exists.",

    "Section 154 of CrPC (Information in case of cognizable offence): In case of cognizable offence, any information relating to the commission of an offence shall be entered in a book to be kept by such officer in such form and manner as the State Government may prescribe.",

    "Section 161 of CrPC (Examination of Witnesses by Police): Any police officer making an investigation into an alleged offence may examine any person supposed to be acquainted with the facts and circumstances of the case.",

    "Section 164 of CrPC (Recording of Confessions and Statements): Any confession made by a person while in custody shall be recorded in writing by a Magistrate. Such confession cannot be used as evidence unless it is corroborated by other evidence.",

    "Section 300 of CrPC (Offence of Murder): Culpable homicide is designated murder, if the act by which the death is caused is done with the intention of causing death, or if it is done with knowledge that the person is by that act in all probability causing death.",

    # Constitution of India
    "Article 14 of Constitution of India (Equality before law): The State shall not deny to any person equality before the law or the equal protection of the laws within the territory of India. Prohibition of discrimination on grounds of religion, race, caste, sex or place of birth.",

    "Article 19 of Constitution of India (Protection of certain rights regarding freedom of speech, etc.): All citizens shall have the right to freedom of speech and expression; freedom to assemble peaceably and without arms; freedom to form associations or unions; freedom to move freely throughout the territory of India.",

    "Article 21 of Constitution of India (Protection of life and personal liberty): No person shall be deprived of his life or personal liberty except according to procedure established by law. This article protects the fundamental right to life and liberty.",

    "Article 32 of Constitution of India (Remedies for enforcement of rights conferred by this Part): The right to move the Supreme Court by appropriate proceedings for the enforcement of the rights conferred by this Part is guaranteed.",

    "Article 51A of Constitution of India (Fundamental Duties): It shall be the duty of every citizen of India to abide by the Constitution and respect its ideals and institutions, the National Flag and the National Anthem; to uphold and protect the sovereignty and integrity of India.",

    # Additional Legal Concepts
    "Mens Rea (Guilty Mind): The mental element or criminal intent required to establish criminal liability. It refers to the intention or knowledge of the accused at the time of committing the act. Presence of Mens Rea is essential to establish most criminal offences.",

    "Actus Reus (Guilty Act): The physical act or illegal act performed. It is the external element of a crime. For example, in theft, the act of taking someone's property without consent constitutes Actus Reus.",

    "Burden of Proof: In criminal law, the burden lies on the prosecution to prove the guilt of the accused beyond a reasonable doubt. In civil cases, the burden of proof is on the preponderance of probabilities.",

    "Statute of Limitations: A law that sets the maximum time period within which legal proceedings can be initiated after a crime has been committed. Different crimes have different limitation periods.",

    "Right to Bail: Every person arrested has the right to bail unless there are strong reasons to believe that the person may flee or tamper with evidence. Bail is a temporary release from custody pending trial.",

    "Habeas Corpus: A writ that orders a person or authority holding someone in custody to bring them before a court and justify the legality of the detention. It is a fundamental protection against unlawful detention.",
]

print(f"Total law chunks: {len(law_chunks)}")
for i, chunk in enumerate(law_chunks[:5]):
    print(f"\nChunk {i+1}: {chunk[:100]}...")


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# Sample Indian Legal Text Chunks (as defined above)
law_chunks = [
    "Section 81 of IPC: Act likely to cause harm, but done without criminal intent, and to prevent other harm. Nothing is an offence by reason of any harm which it may cause to a person who has consented to suffer that harm.",
    "Section 82 of IPC: Nothing is an offence which is done by a person who, at the time of doing it, by reason of immaturity of understanding, is incapable of knowing either the nature of the act, or that he is doing what is wrong or contrary to law.",
    "Section 83 of IPC: Nothing is an offence by reason of any harm which it may cause to a person above eighteen years of age who has consented to suffer that harm.",
    "Section 84 of IPC: Nothing is an offence which is done by a person who, at the time of doing it, by reason of unsoundness of mind, is incapable of knowing the nature of the act.",
    "Section 378 of IPC (Theft): Whoever, intending to take dishonestly any movable property out of the possession of any person without that person's consent, moves that property in order by that movement to effect his purpose, is said to commit theft.",
    "Section 420 of IPC (Cheating): Whoever cheats and thereby dishonestly induces the person deceived to deliver any property to any person shall be punished with imprisonment of either description for a term which may extend to seven years.",
    "Section 498A of IPC (Cruelty): Whoever, being the husband or the relative of the husband of a woman, subjects such woman to cruelty shall be punished with imprisonment for a term which may extend to three years.",
    "Section 41 of CrPC (Arrest without Warrant): An officer in charge of a police station shall, when investigating a case, have the authority to arrest any person against whom a reasonable complaint has been made.",
    "Section 154 of CrPC (Cognizable Offence Information): In case of cognizable offence, any information relating to the commission of an offence shall be entered in a book to be kept by such officer.",
    "Section 164 of CrPC (Recording of Confessions): Any confession made by a person while in custody shall be recorded in writing by a Magistrate.",
    "Article 14 of Constitution of India (Equality before law): The State shall not deny to any person equality before the law or the equal protection of the laws.",
    "Article 19 of Constitution of India (Freedom of Speech): All citizens shall have the right to freedom of speech and expression; freedom to assemble peaceably.",
    "Article 21 of Constitution of India (Right to Life): No person shall be deprived of his life or personal liberty except according to procedure established by law.",
    "Article 32 of Constitution of India (Enforcement of Rights): The right to move the Supreme Court by appropriate proceedings for the enforcement of the rights conferred by this Part.",
    "Mens Rea (Guilty Mind): The mental element or criminal intent required to establish criminal liability. It refers to the intention of the accused at the time of committing the act.",
    "Actus Reus (Guilty Act): The physical act or illegal act performed. It is the external element of a crime.",
    "Burden of Proof: In criminal law, the burden lies on the prosecution to prove the guilt of the accused beyond a reasonable doubt.",
    "Habeas Corpus: A writ that orders a person or authority holding someone in custody to bring them before a court and justify the legality of the detention.",
]

# Initialize embedder and create FAISS index
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(law_chunks)
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings))

# Retrieval function
def retrieve_law_context(query, k=3):
    """Retrieve top k relevant law chunks for a query"""
    query_emb = embedder.encode([query])
    D, I = index.search(np.array(query_emb), k=k)
    results = []
    for idx in I[0]:
        results.append(law_chunks[idx])
    return results

# Test retrieval
test_queries = [
    "What is theft under IPC?",
    "Can police arrest without warrant?",
    "What is the right to equality?"
]

for query in test_queries:
    print(f"\n📋 Query: {query}")
    results = retrieve_law_context(query, k=2)
    for i, result in enumerate(results, 1):
        print(f"   Result {i}: {result[:120]}...")


In [ ]:
import gradio as gr

def lawbot_answer(user_query):
    retrieved_chunks = retrieve_law_context(user_query, k=3)

    results = []
    for i, chunk in enumerate(retrieved_chunks, 1):
        results.append(f"**Result {i}:**\n\n{chunk}\n")

    return "\n---\n".join(results)

def get_ipc_info():
    return """
## Indian Penal Code (IPC)
- **Sections 81-84:** General Exceptions
- **Section 378:** Theft Definition
- **Section 420:** Cheating
- **Section 498A:** Cruelty by Husband/In-laws
    """

def get_crpc_info():
    return """
## Criminal Procedure Code (CrPC)
- **Section 41:** Arrest without Warrant
- **Section 154:** Information in Cognizable Offence
- **Section 161:** Examination of Witnesses
- **Section 164:** Recording of Confessions
    """

def get_constitution_info():
    return """
## Constitution of India
- **Article 14:** Equality Before Law
- **Article 19:** Freedom of Speech & Expression
- **Article 21:** Protection of Life & Liberty
- **Article 32:** Right to Constitutional Remedies
    """

# Create tabbed interface
with gr.Blocks(title="LawBot", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # ⚖️ LawBot - Indian Legal AI Assistant

    Your intelligent guide to Indian law powered by AI
    """)

    with gr.Tabs():
        with gr.TabItem("🔍 Search Query", id=0):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Enter Your Query")
                    query_input = gr.Textbox(
                        label="Legal Query",
                        placeholder="Ask about IPC, CrPC, or Constitution...",
                        lines=4
                    )
                    search_btn = gr.Button("🔍 Search", variant="primary", size="lg")

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📋 Results")
                    query_output = gr.Markdown(label="Search Results")

            search_btn.click(fn=lawbot_answer, inputs=query_input, outputs=query_output)
            query_input.submit(fn=lawbot_answer, inputs=query_input, outputs=query_output)

        with gr.TabItem("📖 IPC Sections", id=1):
            ipc_output = gr.Markdown()
            demo.load(fn=get_ipc_info, outputs=ipc_output)

        with gr.TabItem("⚖️ CrPC Sections", id=2):
            crpc_output = gr.Markdown()
            demo.load(fn=get_crpc_info, outputs=crpc_output)

        with gr.TabItem("🏛️ Constitution", id=3):
            const_output = gr.Markdown()
            demo.load(fn=get_constitution_info, outputs=const_output)

        with gr.TabItem("❓ FAQ", id=4):
            gr.Markdown("""
            ### Frequently Asked Questions

            **Q: Is this legal advice?**
            A: No, this is educational content only.

            **Q: Can I rely on this for court proceedings?**
            A: No, always consult a qualified lawyer.

            **Q: What if I get wrong information?**
            A: Please verify information with official legal sources or consult professionals.

            **Q: How accurate is LawBot?**
            A: LawBot provides information from verified Indian legal texts but may not cover all edge cases.
            """)

    gr.Markdown("""
    ---

    ### ⚠️ Important Disclaimer
    This application is for **educational purposes only**. It does not constitute legal advice.
    For legal matters, please consult with qualified legal professionals.
    """)

demo.launch(share=True, debug=True)
